In [1]:
# Import required libraries for data manipulation and analysis
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import preprocessing

In [2]:
#Import RDKit and Mordred libraries
from rdkit import Chem
from rdkit.Chem import Draw
from mordred import Calculator, descriptors

In [3]:
#Allows figures to be visualized in jupyter notebook
%matplotlib inline

In [4]:
# Sets Pandas Display to Monitor Code
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 200)

#Remove zero varience
def remove_zero_varience(values):
   sel = VarianceThreshold()
   _ = sel.fit(values)
   mask = sel.get_support()
   values = values.loc[:,mask] 
   return values

#Get Mordred calcs
def get_Mordred(data_input):
    # Assigns Reactants Mordred Info
    reactants = data_input['Substrate']
    
    reactants_mol_list = []
    for inChi_reactants in reactants:
      reactants_mol = Chem.MolFromInchi(inChi_reactants)
      reactants_mol_list.append(reactants_mol)

    # Puts reactants into Pandas Type
    reactant_data = []
    reactant_data = calc.pandas(reactants_mol_list)
       
    #Joins Mordred parameters with experimental, atomic charges, and JChem for Excel parameters
    add_reactants = pd.concat((data_input, reactant_data), axis=1)
    
    #Force any non-numeric entries as NaN and replace them with 0
    int_data = add_reactants.apply(pd.to_numeric, errors='coerce')
    
    output = int_data.fillna(0)#, inplace=True)

    return output


In [5]:
# Create Mordred Calculator
calc = Calculator(descriptors, ignore_3D=True)

In [6]:
# Read Training/Test data input File
dataset = pd.read_csv('BorylationTrainingTest 9-26-24.csv')
validation = pd.read_csv('validation8-26-24.csv')
data = pd.concat([dataset, validation], axis=0)
type_data = pd.read_csv('inchis_type9-27-24.csv')
data = data.reset_index(drop=True)

In [7]:
#group the compounds by numbers
data['grouped'] = data.groupby('Substrate', sort=False).ngroup()
data[['Substrate','grouped']]

,Substrate,grouped
0,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
1,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
2,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
3,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
4,"InChI=1S/C7H15N/c1-2-8-6-4-3-5-7-8/h2-7H2,1H3",1
...,...,...
1103,"InChI=1S/C7H9NO/c1-9-7-4-2-3-6(8)5-7/h2-5H,8H2...",210
1104,"InChI=1S/C7H9NO/c1-9-7-4-2-3-6(8)5-7/h2-5H,8H2...",210
1105,"InChI=1S/C7H9NO/c1-9-7-4-2-3-6(8)5-7/h2-5H,8H2...",210
1106,"InChI=1S/C7H9NO/c1-9-7-4-2-3-6(8)5-7/h2-5H,8H2...",210


In [8]:
data = get_Mordred(data)

100%|██████████| 1108/1108 [01:50<00:00, 10.00it/s]


In [9]:
d = preprocessing.normalize(data)
scaled_df = pd.DataFrame(d)
scaled_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,...,1581,1582,1583,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1603,1604,1605,1606,1607,1608,1609,1610,1611,1612,1613,1614,1615,1616,1617,1618,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1630,1631,1632,1633,1634,1635,1636,1637,1638,1639,1640,1641,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680
0,0.0,0.0,0.000082,0.000245,0.000041,0.000041,0.001761,0.000028,0.000418,0.030054,0.058889,0.000243,0.000196,0.000309,0.000243,0.000196,0.000297,-1.384616e-07,-2.361487e-08,-4.157484e-06,0.000001,-3.202675e-06,-7.460420e-06,1.329338e-06,-0.000023,0.000008,-3.639469e-07,-0.000010,0.000003,-9.001434e-06,-0.000025,0.000005,0.000082,0.006128,0.000327,0.000286,0.000000,0.000000,0.000000,0.000000,0.000049,0.000046,0.000103,0.000327,0.001144,0.000496,0.000286,0.000561,0.000000,0.000000,0.0,0.000000,0.000000,0.008580,0.000000,0.002147,0.000269,0.000233,0.000634,0.000490,0.001577,0.000000,0.000204,0.000302,0.003432,0.000041,0.000000,0.000000,0.000202,0.000210,0.0,0.000000,0.000389,0.000077,0.000154,0.000389,0.000049,0.000118,0.000109,0.000014,0.000031,0.000878,0.000110,0.000116,0.000000,0.000000,0.001062,0.000327,0.0,0.0,0.000000,0.000735,0.0,0.000327,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000204,0.000029,0.000138,0.001595,0.000000,0.000000,0.000020,0.000009,0.000005,0.000003,0.000002,0.000002,0.0,0.0,0.0,0.0,0.000003,0.000002,0.000001,8.171236e-07,7.565959e-07,8.337996e-07,0.0,0.0,0.0,0.0,0.000008,0.000286,0.000163,0.000031,0.000018,0.006003,0.000156,0.000286,0.000135,0.000159,0.000184,0.000210,0.000235,0.000261,0.000287,0.000312,0.000338,0.002734,0.000111,0.0,0.000150,0.0,0.000195,0.0,0.000242,0.0,0.000291,0.001315,0.004663,0.000179,0.003432,0.000204,0.001062,0.000981,0.000143,0.000092
1,0.0,0.0,0.000082,0.000245,0.000041,0.000041,0.001761,0.000028,0.000418,0.030054,0.058889,0.000243,0.000196,0.000309,0.000243,0.000196,0.000297,1.264091e-07,1.017319e-08,-2.357974e-06,0.000001,4.013957e-06,7.473986e-06,-1.730015e-06,-0.000016,0.000008,3.808206e-07,-0.000006,0.000003,1.820388e-06,-0.000009,0.000006,0.000082,0.006128,0.000327,0.000286,0.000000,0.000000,0.000000,0.000000,0.000089,0.000046,0.000103,0.000327,0.000899,0.000496,0.000245,0.000561,0.000000,0.000000,0.0,0.000000,0.000000,0.008580,0.000000,0.002147,0.000269,0.000233,0.000634,0.000490,0.001577,0.000000,0.000204,0.000305,0.003432,0.000000,0.000000,0.000000,0.000202,0.000210,0.0,0.000000,0.000389,0.000077,0.000154,0.000389,0.000049,0.000118,0.000109,0.000014,0.000031,0.000878,0.000110,0.000116,0.000000,0.000000,0.001062,0.000327,0.0,0.0,0.000000,0.000735,0.0,0.000327,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000204,0.000029,0.000138,0.001595,0.000000,0.000000,0.000020,0.000009,0.000005,0.000003,0.000002,0.000002,0.0,0.0,0.0,0.0,0.000003,0.000002,0.000001,8.171238e-07,7.565961e-07,8.337998e-07,0.0,0.0,0.0,0.0,0.000008,0.000286,0.000163,0.000031,0.000018,0.006003,0.000156,0.000286,0.000135,0.000159,0.000184,0.000210,0.000235,0.000261,0.000287,0.000312,0.000338,0.002734,0.000111,0.0,0.000150,0.0,0.000195,0.0,0.000242,0.0,0.000291,0.001315,0.004663,0.000179,0.003432,0.000204,0.001062,0.000981,0.000143,0.000092
2,0.0,0.0,0.000082,0.000245,0.000041,0.000041,0.001761,0.000028,0.000418,0.030054,0.058889,0.000243,

In [10]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2)
tsne_result = tsne.fit_transform(scaled_df)

df = pd.DataFrame(tsne_result , columns=['x', 'y'])


In [11]:
df['type'] = type_data['type'].copy()
df

,x,y,type
0,-18.331320,-22.461189,aliphatic
1,-18.331459,-22.461065,aliphatic
2,-18.330410,-22.458050,aliphatic
3,-18.331457,-22.461065,aliphatic
4,3.127728,-30.674297,aliphatic- nitrogen
...,...,...,...
1103,13.755845,-15.388037,NaN
1104,13.755293,-15.388517,NaN
1105,13.757907,-15.384709,NaN
1106,13.759583,-15.378253,NaN


In [12]:
df['group'] = data['grouped'].copy()
df

,x,y,type,group
0,-18.331320,-22.461189,aliphatic,0
1,-18.331459,-22.461065,aliphatic,0
2,-18.330410,-22.458050,aliphatic,0
3,-18.331457,-22.461065,aliphatic,0
4,3.127728,-30.674297,aliphatic- nitrogen,1
...,...,...,...,...
1103,13.755845,-15.388037,NaN,210
1104,13.755293,-15.388517,NaN,210
1105,13.757907,-15.384709,NaN,210
1106,13.759583,-15.378253,NaN,210


In [13]:
#define aggregate function
agg_functions = {'type': 'first', 'x': 'mean','y':'mean' }

dfAgg = df.groupby(df['group']).aggregate(agg_functions)
dfAgg 

,type,x,y
group,,,
0,aliphatic,-18.331161,-22.460342
1,aliphatic- nitrogen,3.127766,-30.674294
2,aliphatic,1.171430,-29.429689
3,ED aromatic,35.956333,-9.834714
4,pyridine and pyrimidine,15.471419,-21.181194
...,...,...,...
206,NaN,-17.108368,34.014061
207,NaN,15.474030,-3.650591
208,NaN,-36.298210,11.526303


In [14]:
dfAT= dfAgg[dfAgg.index < 199]
dfAT

,type,x,y
group,,,
0,aliphatic,-18.331161,-22.460342
1,aliphatic- nitrogen,3.127766,-30.674294
2,aliphatic,1.171430,-29.429689
3,ED aromatic,35.956333,-9.834714
4,pyridine and pyrimidine,15.471419,-21.181194
...,...,...,...
194,furan,8.406651,11.115901
195,furan,-6.200802,27.251619
196,thiophene,-6.884751,27.779070


In [15]:
dfAV= dfAgg[dfAgg.index > 198]
dfAV = dfAV.reset_index()
dfAV

,group,type,x,y
0,199,NaN,2.149180,28.552006
1,200,NaN,-36.034126,31.100269
2,201,NaN,-41.874088,32.263699
3,202,NaN,43.117260,-21.483002
4,203,NaN,-32.522030,38.573177
...,...,...,...,...
7,206,NaN,-17.108368,34.014061
8,207,NaN,15.474030,-3.650591
9,208,NaN,-36.298210,11.526303
10,209,NaN,18.119133,5.007242


In [16]:
aliphatic = dfAT.loc[dfAT.type == 'aliphatic']
ali_nit = dfAT.loc[dfAT.type == 'aliphatic- nitrogen']
ali_oxy = dfAT.loc[dfAT.type == 'aliphatic- oxygen']
aromatic = dfAT.loc[dfAT.type == 'aromatic']
benzofuran = dfAT.loc[dfAT.type == 'benzofuran']
benzoimidazole = dfAT.loc[dfAT.type == 'benzoimidazole']
benzooxazole = dfAT.loc[dfAT.type == 'benzooxazole']
benzothiophene = dfAT.loc[dfAT.type == 'benzothiophene']
EDG_aro = dfAT.loc[dfAT.type == 'ER aromatic']
EWG_aro = dfAT.loc[dfAT.type == 'ED aromatic']
furan = dfAT.loc[dfAT.type == 'furan']
indole = dfAT.loc[dfAT.type == 'indole']
pyr = dfAT.loc[dfAT.type == 'pyridine and pyrimidine']
pyrrole = dfAT.loc[dfAT.type == 'pyrrole']
pyrrolopyridine = dfAT.loc[dfAT.type == 'pyrrolopyridine']
quinoline = dfAT.loc[dfAT.type == 'quinoline']
thiophene = dfAT.loc[dfAT.type == 'thiophene']



In [18]:
# plot
fig = plt.figure(figsize=(8, 8), dpi = 1200)
ax = fig.add_subplot(111, )

#aliphatic = blue
plt.plot(aliphatic['x'],aliphatic['y'], 'o', ms=11, c='#1f77b4',alpha = 0.25 )
plt.plot(ali_nit['x'],ali_nit['y'], 'o', ms=11, c='#1f77b4',alpha = 0.50 )
plt.plot(ali_oxy['x'],ali_oxy['y'], 'o', ms=11, c='#1f77b4',alpha = 0.9 )

#aromatic = purple
plt.plot(aromatic['x'],aromatic['y'], 'o', ms=11, c='#9467bd',alpha = 0.15 )
plt.plot(EDG_aro['x'],EDG_aro['y'], 'o', ms=11, c='#9467bd',alpha = 0.5 )
plt.plot(EWG_aro['x'],EWG_aro['y'], 'o', ms=11, c='#9467bd',alpha = 0.9 )

#aromatic cyclic nitrogen = green
plt.plot(benzoimidazole['x'],benzoimidazole['y'], 'o', ms=11, c='#2ca02c',alpha = 0.06 )
plt.plot(indole['x'],indole['y'], 'o', ms=11, c='#2ca02c',alpha = 0.25 )
plt.plot(pyr['x'],pyr['y'], 'o', ms=11, c='#2ca02c',alpha = 0.4 )
plt.plot(pyrrole['x'],pyrrole['y'], 'o', ms=11, c='#2ca02c',alpha = 0.60 )
plt.plot(pyrrolopyridine['x'],pyrrolopyridine['y'], 'o', ms=11, c='#2ca02c',alpha = 0.70 )
plt.plot(quinoline['x'],quinoline['y'], 'o', ms=11, c='#2ca02c',alpha = .85 )

#aromatic cyclic oxygen = grey
plt.plot(furan['x'],furan['y'], 'o', ms=11, c='#7f7f7f',alpha = 0.15 )
plt.plot(benzofuran['x'],benzofuran['y'], 'o', ms=11, c='#7f7f7f',alpha = 0.5 )
plt.plot(benzooxazole['x'],benzooxazole['y'], 'o', ms=11, c='#7f7f7f',alpha = 0.85 )

#sulur heterocycle = orange
plt.plot(thiophene['x'],thiophene['y'], 'o', ms=11, c='#ff7f0e',alpha = 0.2 )
plt.plot(benzothiophene['x'],benzothiophene['y'], 'o', ms=11, c='#ff7f0e',alpha = 0.7 )


n = ['14','17','24','20','19','18','16','13','23','22','21','15']
     

x_coords = dfAV['x']
y_coords = dfAV['y']
for i,type in enumerate(n):
    x = x_coords[i]
    y = y_coords[i]
    plt.text(x, y, type, fontsize=12)
    
fig.legend(['aliphatic','aliphatic nitrogen','aliphatic oxygen','pyrene','electron rich aromatic','electron poor aromatic',
            'benzoimidazole','indole',
            'pyridine and pyrimidine','pyrrole','pyrrolopyridine','quinoline',
            'furan','benzofuran','benzooxazole','thiophene','benzothiophene',
            ],columnspacing=1,
           bbox_to_anchor=(0.890,0.29), ncol = 3)

#Outlines
#aliphatic = blue
plt.plot(aliphatic['x'],aliphatic['y'], 'o', ms=11, markerfacecolor="None",
         markeredgecolor='#1f77b4', markeredgewidth=1,c='#1f77b4' )
plt.plot(ali_nit['x'],ali_nit['y'], 'o', ms=11, markerfacecolor="None",
         markeredgecolor='#1f77b4', markeredgewidth=1,c='#1f77b4' )
plt.plot(ali_oxy['x'],ali_oxy['y'], 'o', ms=11, markerfacecolor="None",
         markeredgecolor='#1f77b4', markeredgewidth=1,c='#1f77b4' )

#aromatic = purple
plt.plot(aromatic['x'],aromatic['y'], 'o', ms=11, markerfacecolor="None",
         markeredgecolor='#9467bd', markeredgewidth=1,c='#9467bd' )
plt.plot(EDG_aro['x'],EDG_aro['y'], 'o', ms=11, markerfacecolor="None",
         markeredgecolor='#9467bd', markeredgewidth=1,c='#9467bd' )
plt.plot(EWG_aro['x'],EWG_aro['y'], 'o', ms=11, markerfacecolor="None",
         markeredgecolor='#9467bd', markeredgewidth=1,c='#9467bd' )

#aromatic cyclic nitrogen
plt.plot(benzoimidazole['x'],benzoimidazole['y'], 'o', ms=11, markerfacecolor="None",
         markeredgecolor='#2ca02c', markeredgewidth=1,c='#2ca02c' )
plt.plot(indole['x'],indole['y'], 'o', ms=11, markerfacecolor="None",
         markeredgecolor='#2ca02c', markeredgewidth=1,c='#2ca02c' )
plt.plot(pyr['x'],pyr['y'], 'o', ms=11, markerfacecolor="None",
         markeredgecolor='#2ca02c', markeredgewidth=1,c='#2ca02c' )
plt.plot(pyrrole['x'],pyrrole['y'], 'o', ms=11, markerfacecolor="None",
         markeredgecolor='#2ca02c', markeredgewidth=1,c='#2ca02c' )
plt.plot(pyrrolopyridine['x'],pyrrolopyridine['y'], 'o', ms=11, markerfacecolor="None",
         markeredgecolor='#2ca02c', markeredgewidth=1,c='#2ca02c' )
plt.plot(quinoline['x'],quinoline['y'], 'o', ms=11, markerfacecolor="None",
         markeredgecolor='#2ca02c', markeredgewidth=1,c='#2ca02c' )

#aromatic cyclic oxygen
plt.plot(furan['x'],furan['y'], 'o', ms=11, markerfacecolor="None",
         markeredgecolor='#7f7f7f', markeredgewidth=1,c='#7f7f7f' )
plt.plot(benzofuran['x'],benzofuran['y'], 'o', ms=11, markerfacecolor="None",
         markeredgecolor='#7f7f7f', markeredgewidth=1,c='#7f7f7f' )
plt.plot(benzooxazole['x'],benzooxazole['y'], 'o', ms=11, markerfacecolor="None",
         markeredgecolor='#7f7f7f', markeredgewidth=1,c='#7f7f7f' )

#sulur heterocycle
plt.plot(thiophene['x'],thiophene['y'], 'o', ms=11, markerfacecolor="None",
         markeredgecolor='#ff7f0e', markeredgewidth=1,c='#ff7f0e' )
plt.plot(benzothiophene['x'],benzothiophene['y'], 'o', ms=11, markerfacecolor="None",
         markeredgecolor='#ff7f0e', markeredgewidth=1,c='#ff7f0e' )



ax.set(ylim=(-70, 50))
ax.set_xlabel('t-SNE 1', fontsize=14)
ax.set_ylabel('t-SNE 2', fontsize=14)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

plt.show()
fig.savefig('TSNE9-27.tiff', bbox_inches='tight')